# Commodity Price Forecasting with Supervised Machine Learning

## Getting started

- #### Introduction

    - In this project, a number of tree-based regressors are deployed to predict the monthly price direction and concurrent median spot price of a chosen commodity contract over a select period of time (the client will need to specify the type of contract or averaging method across contracts that should be used). 


- #### Data compatibility

    - This code will work for any market data ingested via Barchart to construct the target feature as well either Barchart or Oilworld physical contract data for input features. 

- #### Exploratory data analysis

    - Standard pandas profiling reports are provided along with basic summary statistics to inform skew and missingness in the data.


- #### Data wrangling and pre-processing

    - The current approach converts time series data into a usable format for a supervised machine learning problem and applies the following standard transformations:
       - Handling missing data (represented as NaNs or zeros) using forward-backfill mean imputation. This method can also be substituted for general mean imputation if required
       - Removal of data skew using log transformation
       - Normalization with sklearn MinMax scaling


- #### Feature engineering

    - Automated feature engineering is included in the template of this code. Examples are also provided for information that may be helpful to engineer into additional input features (consult with your CST, the client, and potentially use ENS Navigator to set up expert interviews for identifying targets specific to your use case). 

- #### Feature transformation

    - The option is provided to apply dimensionality reduction using the PCA algorithm from sklearn. The number of dimensions to be used should be specified by the user.


- #### Feature selection 

    - Feature selection defaults to the Boruta method and reverts to F-test selection if no features are initially chosen. Setting a lower iteration rate may also help to reduce run-time and expand the selection, but may also reduce model accuracy.

- #### Model selection and evaluation approach

    - A tree-based model is required to run the code (any type of ensemble or boosted-tree model is compatible but may require additional updates to params grid and confidence interval generator). Currently the model will default to sklearn's RandomForestRegressor.

    - Backtesting is applied using a time series GridSearch expanding window cross-validation approach - the training period must be specified by the user. The model will train up to each point in an expanding window and then make an out-of-sample prediction. These out-of-sample predictions are then collected into a final dataframe along with confidence intervals (currently generated by forestci). This dataframe is then further transformed to convert predictions from % difference between current and future median contract price into spot price predictions.

    - The model is evaluation using standard regression metrics (MAE, MAPE) as well as directional accuracy (precision, recall, and F1 scores). For directional accuracy the option is also provided for both binary and multi-class confusion matrices; the client will need to provide information on the cut-off points to be used for constructing classes. 


- #### Feature importance

    - A gini-importance score is used to rank and plot feature importances (using sklearn package)


- #### Final output and visualization

    - The final predictions, confidence intervals, and information related to actual price as well as % change in price are saved in both .csv and .xlsx format. 

    - Additionally, an ouptut file is provided in both .csv and .xlsx format for populating an MVP PowerBI Dashboard. In order to fully utilize the dashboard, output may also be required from the ACRE Commodity FX hedging optimization support tool.

    - A version of the dashboard is also in development that will only require price forecasting output from this notebook.

## How to run this notebook

- All code boxes requring variable or filename entry will be marked with three stars *** at the top and three dots ... to represent areas where more than one variable may be specified. Please don't forget to remove these before running the code.
- Before running the code, make sure to first run package import and all helper functions at the bottom of the notebook.

### Import packages and input display settings

In [ ]:
# Import packages to process data + run/evaluate the model 
from pathlib import Path
from datetime import date
from time import time
import pandas as pd
import numpy as np
import scipy
from pandas_profiling import ProfileReport
import forestci as fci
from boruta import BorutaPy
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import recall_score, make_scorer, accuracy_score, fbeta_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.exceptions import NotFittedError, DataConversionWarning
from typing import Tuple, List, Dict
from talib import RSI, BBANDS, MACD
import copy
import warnings

In [ ]:
# Pretty display for notebooks
%matplotlib inline
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Ignore irrelevant warning messages
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Set figure size for seaborn plots
sns.set(rc={'figure.figsize':(11, 4)})
sns.set_style('whitegrid')
idx = pd.IndexSlice

## Exploring the Data

### Import the Data

In [ ]:
***
# Load Barchart market data for each set of commodities contracts to be used (these may be used for either input features or target)
# This data should be accesible in csv format through McKinsey ACRE license with Barchart and must be uploaded into your Jupyter notebook environment in order to run the code

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    _INSERT VARIABLE NAME HERE_ = pd.read_csv("_INSERT FILE NAME HERE_.csv")
    _INSERT VARIABLE NAME HERE_.name = "_INSERT VARIABLE NAME HERE_"
    
    ...
    ...
    
    ...
    ...
                                                  
###Examples:
#     Soybean_Oil_US = pd.read_csv("Soybean_oil_US.csv")
#     Soybean_Oil_US.name = "Soybean_Oil_US"

#     CPO_USD_MDEX_Malaysia = pd.read_csv("CPO_USD_MDEX_Malaysia.csv")
#     CPO_USD_MDEX_Malaysia.name = "CPO_USD_MDEX_Malaysia"

#     Indian_Rupee_USD = pd.read_csv("Indian_Rupee_USD.csv")
#     Indian_Rupee_USD.name = "Indian_Rupee_USD"

In [ ]:
***
# Load Oilworld physical contract data
# This data should be accesible in xlsx format through McKinsey ACRE license with Oilworld and must be uploaded into your Jupyter notebook environment in order to run the code

Oilworld_data = pd.read_excel("_INSERT FILE NAME HERE_.xlsx")
Oilworld_data['MONTHLY PRICES'] = Oilworld_data['MONTHLY PRICES'].apply(str)

### Summary statistics and analysis

#### Barchart data

In [ ]:
***
#run profile report for any of the given data - run separately for each dataset from Barchart
profile = ProfileReport(_INSERT VARIABLE NAME HERE_, title="Barchart Pandas Profiling Report", explorative=True)
profile.to_widgets()

###Example
# profile = ProfileReport(CPO_USD_MDEX_Malaysia, title="Barchart Pandas Profiling Report", explorative=True)

#### Oilworld data

In [ ]:
profile = ProfileReport(Oilworld_data, title="Oilword Pandas Profiling Report", explorative=True)
profile.to_widgets()

#### Target Feature Exploration + Analysis

In [ ]:
***
target_data = _INSERT TARGET VARIABLE NAME HERE_.copy()
target_data = target_data.set_index('timestamp')

###Example
# target_data = CPO_USD_MDEX_Malaysia.copy()

In [ ]:
#Take the contract closest to expiration each month
target_data = find_closest_to_spot(target_data)

In [ ]:
###TARGET DEFINITION

###OPTION 1
#% change between the current price and 1-3 month median and 4-6 month median for the next 6 months of selected futures contract closest to spot price

# Calculate median 1-3 
target_data['Median_1_3'] = target_data['settle'].rolling(window=3).median()
target_data['Median_1_3'] = target_data['Median_1_3'].shift(-2)

# Calculate median 4-6 
target_data['Median_4_6'] = target_data['settle'].shift(-3).rolling(window=3).median()

# Calculate % change column between current price and 1-3 month median, then do another column for 4-6 month
target_data['%_diff_price_current_vs_Median_1_3'] = (target_data['Median_1_3'] - target_data['settle']) / target_data['settle']
target_data['%_diff_price_current_vs_Median_4_6'] = (target_data['Median_4_6'] - target_data['settle']) / target_data['settle']

In [ ]:
# Use settle price to determine summary statistics
prices = target_data.settle

# Minimum price of the data
minimum_price = np.amin(prices)

# Maximum price of the data
maximum_price = np.amax(prices)

# Mean price of the data
mean_price = np.mean(prices)

# Median price of the data
median_price = np.median(prices)

# Standard deviation of prices of the data
std_price = np.std(prices)


# Show the calculated statistics
print("Statistics for CPO settle price closest to spot:\n")
print("Minimum price: ${:,.2f}".format(minimum_price))
print("Maximum price: ${:,.2f}".format(maximum_price))
print("Mean price: ${:,.2f}".format(mean_price))
print("Median price ${:,.2f}".format(median_price))
print("Standard deviation of prices: ${:,.2f}".format(std_price))

#### Price distribution analysis
Visualize price distribution, derive percentiles and then use this to generate target variable

In [ ]:
#Histogram of commodity futures contract prices closest to expiration for across entire time period
hist = target_data.settle.hist(density=True)
plt.title("Histogram of Commodity Price Distribution")
plt.xlabel("Price")
plt.ylabel("Frequency")

import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
  
# Fit a normal distribution to the data:
# mean and standard deviation
mu, std = norm.fit(target_data.settle) 
  
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
  
plt.plot(x, p, 'k', linewidth=2)
plt.show()

In [ ]:
#Density plot of commodity futures contract price distribution by month
kde = target_data.settle.plot(kind='kde')
plt.title("Density plot of Commodity Price Distribution by Month")
plt.xlabel("Price")
plt.ylabel("Probability Density")

In [ ]:
#Density plot of commodity futures contract price distribution Z-score by month
target_data['Zscore'] = target_data.groupby('month').settle.apply(lambda x: x.div(x.mean()))
target_data.groupby('month').Zscore.plot.kde()
plt.title("Density plot of Z-score of Commodity Price Distribution")
plt.xlabel("Z Score")
plt.ylabel("Probability Density")

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Percentile plot of commodity price distribution over total time period
    g = target_data
    i = g['settle'].quantile([0.05, 0.25, 0.5, 0.9])
    j = g['settle'].agg(['min', 'max'])
    pd.concat([i, j], 1)
    i.T.plot(subplots=True)
    plt.title("Percentile plot of Commodity Price Distribution")
    plt.xlabel("Percentile")
    plt.ylabel("Price")
    plt.show()

    # Percentile plot of commodity price distribution Z-score over total time period
    i = g['Zscore'].quantile([0.05, 0.25, 0.5, 0.9])
    j = g['Zscore'].agg(['min', 'max'])
    pd.concat([i, j], 1)
    i.T.plot(subplots=True)
    plt.title("Percentile plot of Z-Score of Commodity Price Distribution")
    plt.xlabel("Percentile")
    plt.ylabel("Z-Score")
    plt.show()

In [ ]:
# Subset actual price and median 1-3, 4-6 in order to convert % back to price and evaluate MAPE after running the model
target_data_select = target_data[['price_date', 'month', 'year', 'settle', 'Median_1_3', 'Median_4_6']]
target_data_select['price_date'] = target_data_select.price_date - pd.offsets.MonthBegin(1)
target_data_select = target_data_select.set_index('price_date')
target_data_select.index = pd.DatetimeIndex(target_data_select.index)

In [ ]:
# Subset target_data to only include target features
target_data = target_data.drop(columns=['price_date', 'settle', 'Zscore'])
target_data['day'] = 1
target_data['price_date'] = pd.to_datetime(target_data[['year', 'month', 'day']])
target_data = target_data.drop(columns='day')
target_data = target_data.set_index('price_date')                                                                       

## Preparing the Data

### Pre-processing

#### Barchart alternative plant-based commodities, other related commodities, and FX

In [ ]:
***
# Run helper function on each commodity dataframe to select only closest contract to spot price per month 
# Please refer to  helper function method for 'find_closest_to_spot' for other options if the client requests a different approach)

_INSERT VARIABLE NAME HERE_ = find_closest_to_spot(_INSERT VARIABLE NAME HERE_)
...
...


###Examples:
# preprocessed_Soybean_Oil_US = find_closest_to_spot(Soybean_Oil_US)
# preprocessed_CPO_USD_MDEX_Malaysia = find_closest_to_spot(CPO_USD_MDEX_Malaysia) 
# preprocessed_Ethanol_US = find_closest_to_spot(Ethanol_US)
# preprocessed_Indian_Rupee_USD = find_closest_to_spot(Indian_Rupee_USD)

In [ ]:
***
#SET DATE CUTOFF - Run pre-processing on each commodity dataframe to select only closest contract to spot price per month

_INSERT VARIABLE NAME HERE_ = preprocess_commodity_date(_INSERT VARIABLE NAME HERE_, _INSERT DATE HERE_)
...
...

###Examples:
# preprocessed_Soybean_Oil_US = preprocess_commodity_date(preprocessed_Soybean_Oil_US, '2010-01-01')
# preprocessed_CPO_USD_MDEX_Malaysia = preprocess_commodity_date(preprocessed_CPO_USD_MDEX_Malaysia, '2010-01-01') 
# preprocessed_Ethanol_US = preprocess_commodity_date(preprocessed_Ethanol_US, '2010-01-01')
# preprocessed_Indian_Rupee_USD = preprocess_commodity_date(preprocessed_Indian_Rupee_USD, '2010-01-01')

Data for some contracts is not always present on a monthly basis - however it will consistently be missing in the same months across all years for any given contract

In [ ]:
***
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    _INSERT VARIABLE NAME HERE_ = crop_barchart_data(_INSERT VARIABLE NAME HERE_)
    ...
    ...
    
###Examples
# preprocessed_Soybean_Oil_US = crop_barchart_data(preprocessed_Soybean_Oil_US)
# preprocessed_CPO_USD_MDEX_Malaysia = crop_barchart_data(preprocessed_CPO_USD_MDEX_Malaysia)
# preprocessed_Ethanol_US = crop_barchart_data(preprocessed_Ethanol_US)
# preprocessed_Indian_Rupee_USD = crop_barchart_data(preprocessed_Indian_Rupee_USD)

#### OIlworld alternative plant-based commodities

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    Oilworld_data['month'] = [x.split('.')[0] for x in Oilworld_data['MONTHLY PRICES']]
    Oilworld_data['month'] = Oilworld_data['month'].astype(int)

    Oilworld_data['year'] = [x.split('.')[1] for x in Oilworld_data['MONTHLY PRICES']]
    Oilworld_data['year'] = Oilworld_data['year'].astype(int)

    Oilworld_data['day'] = 1
    Oilworld_data['date'] = pd.to_datetime(Oilworld_data[['year', 'month', 'day']])

    Oilworld_data = Oilworld_data.set_index('date')

### Renaming data columns

In [ ]:
***
# rename price col for each barchart commodity dataframe
_INSERT VARIABLE NAME HERE_ = _INSERT VARIABLE NAME HERE_.rename(columns={'settle': '---INSERT VARIABLE NAME HERE---'})
...
...

###Examples
# preprocessed_Soybean_Oil_US = preprocessed_Soybean_Oil_US.rename(columns={'settle': 'Soybean oil futures contract, USD, closest to spot price for year-month'})
# preprocessed_CPO_USD_MDEX_Malaysia = preprocessed_CPO_USD_MDEX_Malaysia.rename(columns={'settle': 'Crude palm oil futures contract, USD (MDEX), closest to spot price for year-month'})
# preprocessed_Ethanol_US = preprocessed_Ethanol_US.rename(columns={'settle': 'Ethanol futures contract, USD, closest to spot price for year-month'})
# preprocessed_Indian_Rupee_USD = preprocessed_Indian_Rupee_USD.rename(columns={'settle': 'USD-INR FX Future, closest contract to spot price'})

NOTE: For Oilworld data we are using the lowest representative asking prices for nearest forward shipment in bulk (excl. import duty if any) US-$/T

### Combining data
Here a number of aggregation transformations are done in order to combine the data across multiple input sources into a master table that will be used for training and runnin the model.

In [ ]:
***
# MASTER_TABLE: merge barchart features into oilworld dataframe on month and year to create master table
master_table = Oilworld_data.merge(---INSERT VARIABLE NAME HERE---, on=['month', 'year'], how='left')
master_table = master_table.merge(---INSERT VARIABLE NAME HERE---, on=['month', 'year'], how='left')
...
...
master_table = master_table.merge(target_data, on=['month', 'year'], how='left')

###Example
# master_table = Oilworld_data.merge(preprocessed_Soybean_Oil_US, on=['month', 'year'], how='left')
# master_table = master_table.merge(preprocessed_Ethanol_US, on=['month', 'year'], how='left')
# master_table = master_table.merge(preprocessed_Indian_Rupee_USD, on=['month', 'year'], how='left')
# master_table = master_table.merge(preprocessed_CPO_USD_MDEX_Malaysia, on=['month', 'year'], how='left')
# master_table = master_table.merge(target_data, on=['month', 'year'], how='left')

In [ ]:
master_table['date'] = pd.to_datetime(master_table[['year', 'month', 'day']])
master_table = master_table.set_index('date').drop(columns=['MONTHLY PRICES'])
master_table = master_table.reset_index().set_index('month').reset_index().set_index('year').reset_index().set_index('date')
master_table = master_table.drop(columns=['day']).sort_values(by='date', ascending=True).loc[master_table.year > 2011]

### Handling NaNs and Zeros

In [ ]:
# Agreed that going forward we will implement mean imputation (forward/back-fill mean) for missing month-years
master_table = pd.concat([master_table.ffill(), master_table.bfill()]).groupby(level=0).mean()

# Alternative more simple approach for mean imputation
# master_table = master_table.fillna(0)
# master_table = handle_zero(master_table)

### Transforming Skewed Continuous Features
A dataset may sometimes contain at least one feature whose values tend to lie near a single number, but will also have a non-trivial number of vastly larger or smaller values than that single number.  Algorithms can be sensitive to such distributions of values and can underperform if the range is not properly normalized.

For highly-skewed feature distributions, it is common practice to apply a <a href="https://en.wikipedia.org/wiki/Data_transformation_(statistics)">logarithmic transformation</a> on the data so that the very large and very small values do not negatively affect the performance of a learning algorithm. Using a logarithmic transformation significantly reduces the range of values caused by outliers. Care must be taken when applying this transformation however: The logarithm of `0` is undefined, so we must translate the values by a small amount above `0` to apply the the logarithm successfully.

In [ ]:
***
# Log-transform the skewed features
skewed = [
    "---INSERT VARIABLE NAME HERE---"
    ...
    ...
]

features_log_transformed = pd.DataFrame(data = master_table)
features_log_transformed[skewed] = master_table[skewed].apply(lambda x: np.log(x + 1))

###Example:
# skewed = [
#     "Soybean oil,U.S.,fob Decatur",
#     "Palm stearin RBD, Mal,cif Rott",
#     "PFAD, Mal fob",
#     "Palmkern oil,Mal/Indo,cif Rott",
#     "Soybean oil futures contract, USD, closest to spot price for year-month",
#     "Ethanol futures contract, USD, closest to spot price for year-month",
#     "USD-INR FX Future, closest contract to spot price"
# ]

In [ ]:
#TODO: try further outlier detection/removal here

## Feature Engineering

### Example strategy from expert meetings related to commodity price forecasting for CPO

1. Start with commodity balance sheet items
2. Then move on to political drivers, followed by technical indicators, weather, and FX if time allows
--- 
1. ##### Balance Sheet
    1. Price spread between palm and Argentine soybean oil – FCPO MDEX vs. Chicago contract
    2. Price spread between palm olein (Malaysia OTC) vs. Argentine soybean oil - if less than 100 USD then shift from palm to soybean (close to 50 dollars you can clearly see this shift) , and in the winter months even more (when palm struggles)
    3. Draw between CPO and closing stocks. Production is important but closing stock is key for driving price!
        1. Big destinations (during last 2 years) all run on very tight stocks – India has run on half the stock that it normally does (all the countries actually, all except China) – reason being price went up is that bank limits stay the same
    4. Bulk vs. non-bulk --- helps show demand distribution going out of producing countries (big amount goes to Africa)
    5. Correlation between domestic consumption and price band 
        1. High correlation usually means domestic consumption below 200,000 - 300,000 metric tons
        2. COVID did not have an impact on consumption but added a lot of price volatility to supply - timing of COVID coincided with DCE being closed -- went limit down straight away
        3. Consumption actually went up in 2021, just a bit down in 2020, not much demand destruction overall
    6. FCPO backwardation and contango
        1. High backwardation months you would find strong correlation with price movements 
        2. Alot of interest in palm oil as replacement for gas as a result
        3. When backwardation is steep they’ll go buy the backend of the curve and sell the backend on the gas oil - backwardation is influenced by the palm oil buyers
        4. Forward curve slope -- some markets take advantage of later months, but India etc. markets don’t take advantage of this (those countries don’t have the ability to

2. ##### Political drivers between producing and consuming countries
    1. Indonesia biofuel mandate --- most successful in the world (every ton CPO exported has to attract x) -- this goes into fund that is then used to (google for example B30 – check if you can use this)
        1. The amount of money the fund has (shows how long the program can last) --- price goes up if the program is going well - duty is the function of the price
    3. Malaysia does not  currently have an enforced biofuel mandate
    4. Tariffs can be extracted via export and shipping duties:
        1. Export duties - fixed base, the average price from 20-19th of every month ) - common tender price established in Indonesia daily
        2. You can predict the export duties based on the established daily common tender price ---- CPO, CPKO, etc.
        

3. ##### Technical indicators:
    1. Worth looking into MACD, RSI, and potentially other momentum indicators
    1. Indonesian plantations and Chinese companies primarily look at charts because they don’t have strong/rigorous research desk. This might have a knock-on effect because traders might try to push price up or down because of certain support levels (self-fulfilling effect of companies using same indicators)

4. ##### Weather
    1. Look into precipitation data, especially from Malaysia where more data is available
    2. All the roads etc get blocked in large producing countries when rain is too heavy, so more difficult to extract palm fruit and oil extract rates go down
    3. Annual filings of Wilmar, Cargill, FGB, etc. for oil extraction

5. ##### FX
    1. Not so important (not much volatility, including during COVID)
----------
#### Data sources:
1. Barchart, Oilword, Reuters
2. MPOB – Malaysian Palm Oil Board --- they have a statistics column frommwhich we could create the entire balance sheet
3. SCS surveyor information from Malaysia
4. Bal data for weather and fertilizer data related to palm – Oilworld might have some as well

#### Dropping all unnecessary and heavily correllated target-related columns

In [ ]:
***
# Drop unecessary columns
features_raw = features_log_transformed.drop(columns=['year', 'month'])

# Select features to drop (some need to be renamed, dropping temporarily)
features_raw = features_raw.drop(columns=[
    "---INSERT VARIABLE NAME HERE---",
    ...
    ...
], axis = 1)

###Example:
# features_raw = features_raw.drop(columns=[
#     'Crude palm oil futures contract, USD (MDEX), closest to spot price for year-month'
#     "Palm oil crude, cif Rotterdam"
# ], axis = 1)

In [ ]:
# Drop any duplicates and reset index to run automated feature generation
features_raw = features_raw.reset_index().drop_duplicates(subset=['date'], keep='first').set_index('date')

In [ ]:
***
# Save targets to append later
targets = features_raw.loc["---INSERT TARGET NAME HERE---", ..., ...]

###Example:
# targets = features_raw.loc["%_diff_price_current_vs_Median_1_3", "%_diff_price_current_vs_Median_4_6"]

In [ ]:
# Safeguard: handle any remaining NaNs w/ forward/backfill mean imputation
features_raw = pd.concat([features_raw.ffill(), features_raw.bfill()]).groupby(level=0).mean()

In [ ]:
***
# Drop any remaining featrues that may create target leak
features_raw = features_raw.drop(columns=["---INSERT TARGET NAME HERE---", ..., ...])
features_raw = features_raw.reset_index()
features_raw = features_raw.set_index(pd.DatetimeIndex(features_raw['date']))

###Example:
# features_raw = features_raw.drop(columns=["Palm oil RBD, Mal, fob", "Palm olein RBD, Mal, fob"])

In [ ]:
# Build input list for the lags to be created with feature auto-generation
x_list_input = createList(6)
x_list_input = x_list_input[2:]

In [ ]:
features_raw = calculate_features(features_raw, features=features_raw.columns[1:27], x_list=x_list_input, index_col='date')

In [ ]:
# Re-combine auto-generated features w/ target
features_raw = features_raw.join(targets).drop(columns=['date'])

In [ ]:
# Safeguard: Run forward/backfill mean imputation as a safeguard to account for inf in the data created during automated feature engineering
features_raw = features_raw.replace([np.inf, -np.inf], np.nan)
features_raw = pd.concat([features_raw.ffill(), features_raw.bfill()]).groupby(level=0).mean()

### Normalizing Numerical Features
In addition to performing transformations on features that are highly skewed, it is often good practice to perform some type of scaling on numerical features. Applying a scaling to the data does not change the shape of each feature's distribution; however, normalization ensures that each feature is treated equally when applying supervised learners. Note that once scaling is applied, observing the data in its raw form will no longer have the same original meaning, as exampled below.

We will use [`sklearn.preprocessing.MinMaxScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) for this.

In [ ]:
***
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)

numerical = [
    "---INSERT VARIABLE NAME HERE---",
    ...
    ...
]

###Example: 
# numerical = [
#     'Crude palm oil futures contract, USD (MDEX), closest to spot price for year-month',
#     "Palm oil crude, cif Rotterdam"
# ]

### Example for fast implementation - Make sure to only run on input feature columns (easiest to use iloc with big datasets such as example below)
# numerical = features_raw.iloc[:,:2470].columns

features_log_minmax_transform = pd.DataFrame(data = features_raw)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_raw[numerical])

# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))

# Automated Modeling Pipeline

## Evaluating Model Performance

The following supervised regressor algorithms have been chosen for the initial model:

**Decision Tree**
- Real World Application:
   - Decision Trees are often applied to customer relationship management systems (CRMs). For example, they can be used to investigate the relationships between the customers’ needs and preferences and the success of online shopping (Lee et al. 2007).
- Strengths: 
   - They easily handle feature interactions (one-up over Naive Bayes) and they’re non-parametric, so you don’t have to worry about outliers or whether the data is linearly separable. They’re also fast and scalable, and you don’t have to worry about tuning a bunch of parameters like you do with other classifiers like SVMs. In this case, if the data includes feature interactions that are not linearly-separable, then a Decision Tree will still perform well.
- Weaknessses:
   - One disadvantage is that they don’t support online learning, so you have to rebuild your tree when new examples come on. Another disadvantage is that they easily overfit, but that’s where ensemble methods like random forests (or boosted trees) come in. In this case overfitting may occur, which is why the next algorithm I chose incorporates an ensemble method.
- What makes this model a good candidate for the problem, given what you know about the data?
   - Since Decision Trees can easily handle feature interactions without sacrificing much in speed or cost they seem like a good alternative to Naive Bayes in the event that feature interactions are present in the data.
   
   
**Random Forest**
- Real World Application:
   - Random Forest is used in numerous applications for regression and classification when predicting on large well-established datasets (for example in manufacturing or agriculture where large amounts of data are often collected using sensors)
- Strengths: 
   - Random Forest is suitable for situations when we have a large dataset, and interpretability is not a major concern
- Weaknessses:
   - Since a Random Forest combines multiple decision trees, it becomes more difficult to interpret, however one can take the best trees to build micro-segmentation if required. Decision trees are much easier to interpret and understand. 
- What makes this model a good candidate for the problem, given what you know about the data?
   - Since Random Forest can easily handle large feature sets without sacrificing much in speed or cost they seem like a good alternative to Decision Tree when our feature space is large.
   
   
**Gradient-Boosted Trees**
- Real World Application:
   - Gradient-boosted tree algorithms (eg., XGBoost, LightGBM) are applied to all kinds of use cases where model explainability is less important than accuracy. This is because feature relevance will be harder to interpret and not to scale (for example you will rely on SHAP scores that cannot be used to inform micro-segmentation and rules-based marketing if desired).
- Strengths: 
   - Gradient-boosted trees generally provide higher accuracy models when compared to other tree-based/ensemble approaches
- Weaknessses:
   - Model interpretability becomes more challenging and the model may take longer to train depending on parameters being used
- What makes this model a good candidate for the problem, given what you know about the data?
   - Can be a good way to demonstrate that even better models are possible if accuracy is poor with other tree-based/ensemble methods

### Obtain final features table (includes pre-processed input features as well as all target features)

In [ ]:
#Obtain final features from pre-processing
features_final = features_log_minmax_transform

#Safeguard: Forward/Back-fill mean imputation for any missing values (safeguard for Null/Inf values created post-log-minmax transformations)
features_final = pd.concat([features_final.ffill(), features_final.bfill()]).groupby(level=0).mean()

# Alternative more simple approach for mean imputation
# master_table = master_table.fillna(0)
# master_table = handle_zero(master_table)

### Feature Transformation (Optional)
#### Implementation: PCA  (Dimensionality Reduction)
Now that the data has been scaled to a more normal distribution and has had any necessary outliers removed, I decided to apply PCA to `features_final` to discover which dimensions about the data best maximize the variance of features involved. In addition to finding these dimensions, PCA will also report the explained variance ratio of each dimension — how much variance within the data is explained by that dimension alone. Note that a component (dimension) from PCA can be considered a new "feature" of the space, however it is a composition of the original features present in the data.

When using principal component analysis, one of the main goals is to reduce the dimensionality of the data — in effect, reducing the complexity of the problem. Dimensionality reduction comes at a cost: Fewer dimensions used implies less of the total variance in the data is being explained. Because of this, the cumulative explained variance ratio is extremely important for knowing how many dimensions are necessary for the problem. Additionally, if a signifiant amount of variance is explained by only two or three dimensions, the reduced data can be visualized afterwards.

In [ ]:
***(OPTIONAL)
# Select targets to drop when running PCA
drop_targets = ["---INSERT TARGETS TO DROP HERE---", ..., ...]

In [ ]:
***(OPTIONAL)
# Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components= "---INSERT NUMBER OF PRINCIPAL COMPONENTS TO USE HERE (AS INTEGER)---")

###Example:
# pca = PCA(n_components= 7)

pca.fit(features_final.drop(columns=drop_targets))

# Transform the good data using the PCA fit above
reduced_data = pca.transform(features_final.drop(columns = drop_targets))

# Create a DataFrame for the reduced data
reduced_data = pd.DataFrame(reduced_data, columns = ["---INSERT DIMENSIONS HERE---", ..., ...])

###Example:
# reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4', 'Dimension 5', 'Dimension 6', 'Dimension 7'])

reduced_data.index = features_final.index

# Re-combine targets and display data after applying PCA transformation in 7 dimensions
reduced_data = pd.merge(reduced_data, features_final[targets], left_index=True, right_index=True, how='left')
reduced_data.head()

### Model evaluation metrics

In [ ]:
#Used for quick and dirty gridsearch across multiple algorithms
def regression_results(y_true, y_pred):
    # Regression metrics
#     explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mean_absolute_pct_error=metrics.mean_absolute_percentage_error(y_true, y_pred)
#     mse=metrics.mean_squared_error(y_true, y_pred) 
#     median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
#     r2=metrics.r2_score(y_true, y_pred)
#     print('explained_variance: ', round(explained_variance,4))    
#     print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MAPE: ', round(mean_absolute_pct_error,4))
#     print('MSE: ', round(mse,4))
#     print('RMSE: ', round(np.sqrt(mse),4))

In [ ]:
#used for scoring with gridsearchCV
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    score = mean_squared_error(y_true, y_predict, squared=False)
    # Return the score
    return score

In [ ]:
#used for scoring with gridsearchCV
def rmse(actual, predict):
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    return score

rmse_score = make_scorer(rmse, greater_is_better = False)

### Pre-modelling feature selection
Correlation analysis 
F-Test

In [ ]:
# F-test feature selection - implemented within the modeling pipeline below
from sklearn.feature_selection import f_regression

def select_features(X_train, y_train, X_test):
    # configure to select a subset of features
    fs = SelectKBest(score_func=f_regression, k=5)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    
    return X_train_fs, X_test_fs, fs

Boruta feature selection

In [ ]:
## Boruta feature selection - implemented within the modeling pipeline below
def select_features_boruta(model, X_train, y_train, X_test):
    boruta = BorutaPy(
       estimator = model, 
       n_estimators = 'auto',
       max_iter = 100 # number of trials to perform
    )
    ### fit Boruta (it accepts np.array, not pd.DataFrame)
    boruta.fit(np.array(X_train), np.array(y_train))
    
    # transform train input data
    X_train_fs = boruta.transform(X_train)
    
    # transform test input data
    X_test_fs = boruta.transform(X_test)

    ### print results
    green_area = X_train.columns[boruta.support_].to_list()
    blue_area = X_train.columns[boruta.support_weak_].to_list()
    print('features in the green area:', green_area)
    print('features in the blue area:', blue_area)
    
    return X_train_fs, X_test_fs

### Train and evaluate model

Below function implements TimeSeriesSplit to perform expanding window cross-validation on the training set, then evaluates across multiple metrics on the out of sample test set. This function can be chained to make step-predictions which different specified training_size, n_splits

In [ ]:
***
def run_price_forecasting(select_target:str, features_final:pd.DataFrame, feature_importances_table:pd.DataFrame, train_size:int, overlap_drop:int, num_splits:int, confidence:int):

    #create list of targets not in use to be dropped before doing time series split. Conversion to ML supervised learning training set will then be done on training data sans target
    drop_features = ["---INSERT HERE TARGET FEATURES NOT BEING USED THAT WILL BE DROPPED---"]
    
    #Example:
    # drop_features = ['%_diff_price_current_vs_Median_1', '%_diff_price_current_vs_Median_2', '%_diff_price_current_vs_Median_3']
    
    index = drop_features.index(select_target)
    del drop_features[index]    #removes the selected target from the drop features list
    features_final = features_final.drop(columns=drop_features)
    features_final = features_final.reset_index().drop_duplicates(subset=['date'], keep='first').set_index('date')

    # Create training set
    df_train = features_final[:train_size]

    # Create test set
    df_test = features_final[train_size:] 

    for col in df_train, df_test:
        plt.title('Commodity price forecasting train and test sets', size=20)
        plt.plot(df_train, label='Training set')
        plt.plot(df_test, label='Test set', color='orange')
    plt.show()
    plt.clf()

    #Create X_train, X_test, y_train, y_test
    X_train = df_train.drop(columns=[select_target])
    y_train = df_train[select_target]
    X_test = df_test.drop(columns=[select_target])
    y_test = df_test[select_target]

    # Removing last n rows to prevent target leak with training set (targets are forward looking)
    X_train = X_train.iloc[:-overlap_drop]
    y_train = y_train.iloc[:-overlap_drop]

    # Insert model to be used
    model = "---INSERT MODEL BEING USED HERE---" 
    #Example: model = RandomForestRegressor(bootstrap=True)

    # Insert params grid to be used during gridsearch
    params_search = {"---INSERT PARAMS GRID HERE---"}
    
    #Example:
    #     param_search = { 
    #         'n_estimators': [10, 20, 50],
    #         'max_features': ['auto', 'sqrt', 'log2'],
    #         'max_depth' : [i for i in range(5,15)]
    #     }
    
    print("\n")
    print("...........Now selecting features")
    
    # Feature selection: Boruta, revert to F-test if empty green/blue area
    boruta = BorutaPy(
    estimator = model, 
    n_estimators = 'auto',
    max_iter = 100) # number of trials to perform
    
    boruta.fit(np.array(X_train), np.array(y_train))
    green_area = X_train.columns[boruta.support_].to_list()
    blue_area = X_train.columns[boruta.support_weak_].to_list()
    print('features in the green area:', green_area)
    print('features in the blue area:', blue_area)

    if(len(blue_area) != 0):   
        X_train_fs = X_train[X_train.columns.intersection(blue_area)]
        X_test_fs = X_test[X_test.columns.intersection(blue_area)]
    elif((len(blue_area) == 0) and (len(green_area) != 0)):
        X_train_fs = X_train[X_train.columns.intersection(green_area)]
        X_test_fs = X_test[X_test.columns.intersection(green_area)]
    else:
        # Feature selection: F-test
        X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)  

    print("Currently selecting following number of features:")
    print(pd.DataFrame(X_train_fs).shape[1])

    # Set the gap to be used when performing cross_validation with TimeSeriesSplit from sklearn
    gapper = overlap_drop
    
    # Initialize TimeSeriesSplit for time series cross-validation when performing gridsearch across params (using rmse below due to negative values in target)
    tscv = TimeSeriesSplit(n_splits=num_splits, gap=gapper)
    gsearch = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, scoring = rmse_score)

    print("\n")
    print("...........Now training the model")
    gsearch.fit(X_train_fs, y_train)
    best_score = gsearch.best_score_
    best_model = gsearch.best_estimator_
    y_true = y_test.values

    print("\n")
    print("...........Now predicting results")
    y_pred = best_model.predict(X_test_fs)
    regression_results(y_true, y_pred)
    
    print("\n")
    print("...........Now extracting feature importances")
    
    # Extract the feature importances using .feature_importances_ 
    importances = best_model.feature_importances_
    
    # Get columns to keep and create new dataframe with those only
    if(len(blue_area) != 0):   
        for feature in blue_area:
            feature = str(feature)
        cols = blue_area
        features_df_new = X_train.loc[:,cols]
    elif((len(blue_area) == 0) and (len(green_area) != 0)):
        for feature in green_area:
            feature = str(feature)
        cols = green_area
        features_df_new = X_train.loc[:,cols]
    else:
        cols = fs.get_support(indices=True)
        features_df_new = X_train.iloc[:,cols]
    
    # Save feature importances w/ Gini score in dictionary
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(features_df_new.columns, best_model.feature_importances_):
        feats[feature] = importance #add the name/value pair 
    importance_table = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    
    #Plot Gini importances
    importance_table.sort_values(by='Gini-importance').plot(kind='bar')
    plt.title("Ranking of Gini-importance for selected features")
    
    # Append feature importances to master output list
    feature_importances_table = feature_importances_table.append(importance_table)
    
    # Plot MDI
    std = np.std([tree.feature_importances_ for tree in best_model.estimators_], axis=0)
    forest_importances = pd.Series(importances, index=features_df_new.columns)
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title("Feature importances using MDI")
    ax.set_ylabel("Mean decrease in impurity")
    fig.tight_layout()
    plt.show()
    plt.clf()
    
    print("\n")
    print("...........Now calculating error")

    # Calculate the variance
    variance = fci.random_forest_error(best_model, X_train_fs, X_test_fs)

    # Plot error bars for predicted target using unbiased variance
#     plt.errorbar(y_test, y_pred, yerr=np.sqrt(variance), fmt='o')
#     plt.xlabel('Reported')
#     plt.ylabel('Predicted')
#     plt.show()

    # Prepare new dataframe to store confidence intrervals
    confidence_intervals = pd.DataFrame(variance)
    confidence_intervals = confidence_intervals.rename(columns={confidence_intervals.columns[0]: 'variance'})
    confidence_intervals["predictions"] = y_pred.tolist()
    
    #Set Z Scores for normal dist. based on confidence
    zscore = 1.65 #default 
    if(confidence == 90):
        zscore = 1.65
    elif(select_target == 95):
        zscore = 1.96
    elif(select_target == 99):
        zscore = 2.58

    #DEFINE CI REQUIREMENT
    confidence_intervals['std'] = np.sqrt(confidence_intervals['variance'])
    confidence_intervals['upper_bound'] = confidence_intervals['predictions'] + (confidence_intervals['std'] * zscore)
    confidence_intervals['lower_bound'] = confidence_intervals['predictions'] - (confidence_intervals['std'] * zscore)


    print("\n")
    print("...........Now appending results to final df")
    
    # Create empty dataframe for forecast results to be exported to excel format (for each select target for which the model is run on a given training window this will be updated)
    Forecast_results = pd.DataFrame(X_test_fs)

    #Append predictions
    col_name1 = 'prediction'
    Forecast_results[col_name1] = y_pred.tolist()
    
    #Append actuals
    col_name2 = 'actual'
    Forecast_results[col_name2] = y_test.tolist()

    #Append upper bound
    col_name3 = 'upper_bound'
    Forecast_results[col_name3] = confidence_intervals['upper_bound'].tolist()

    #Append lower bound
    col_name4 = 'lower_bound'
    Forecast_results[col_name4] = confidence_intervals['lower_bound'].tolist()

    #Append number of forecast months
    col_name5 = 'number_of_forecast_month'
    Forecast_results[col_name5] = gapper
    
    #Reduce forecast results for this particular target using this particular training window to just the prediction for first month on out of sample
    Forecast_results = pd.DataFrame(Forecast_results[[col_name1, col_name2, col_name3, col_name4, col_name5]])
    Forecast_results = Forecast_results.iloc[0, :]
    
    return Forecast_results, feature_importances_table

In [ ]:
***
# SET TRAINING WINDOW
training_window = "---INSERT TRAINING WINDOW HERE---" #take first x months as training set
###Example: training_window = 48

# DATE TO FORECAST
forecast_date_index = features_final.index

# SET FORECAST LENGTH
forecast_length = len(forecast_date_index) - training_window

# SET EMPTY SERIES TO APPEND FEATURE IMPORTANCES
complete_feature_importances = pd.DataFrame()
master_feature_list = pd.DataFrame()

###############RUN THE PREDICTIONS
# Create empty dataframe to store all final results for model predicting next 1-3 months median as well as next 4-6
Final_forecast_results = pd.DataFrame()

targets = [{'target': "---INSERT TARGET HERE---", 'offset': "---INSERT OFFSET HERE---"}, ..., ...]
###Example: targets = [{'target':'%_diff_price_current_vs_Median_1_3', 'offset':3},{'target':'%_diff_price_current_vs_Median_4_6', 'offset':6}]


for i in range(forecast_length):
    #TODO: initiate empty dataframe here and for every day append with x=1
    daily_forecast_result = pd.DataFrame()
    decision_month = forecast_date_index[training_window] 
    current_month = 0
    for target in targets:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            #BELOW LINE RUNS THE MODEL - In order to use PCA meta-feature set just replace 'features_final' in the function call with 'reduced_data'
            globals()['Forecast_Results_%s' % target['target'].replace("%","")], complete_feature_importances = run_price_forecasting(target['target'], features_final, feature_importances_table=complete_feature_importances, train_size=training_window, overlap_drop=target['offset'], num_splits=3, confidence=95) 
            master_feature_list = master_feature_list.append(complete_feature_importances)
            
            for month_iter in range(current_month, target['offset']):
                target_month = month_iter + 1
                # prediction  upper_bound  lower_bound
                copy_of_forecast = pd.DataFrame(copy.deepcopy(globals()['Forecast_Results_%s' % target['target'].replace("%","")])).T[['prediction', 'actual', 'upper_bound', 'lower_bound']]
                copy_of_forecast = copy_of_forecast.rename(columns = {
                    "prediction":"forecast_result_{}".format(target_month), 
                    "actual":"actual_{}".format(target_month), 
                    "upper_bound":"forecast_upper_{}".format(target_month), 
                    "lower_bound":"forecast_lower_{}".format(target_month),
                    })
                daily_forecast_result = pd.concat([daily_forecast_result, copy_of_forecast], axis=1)

            current_month = target['offset']
    daily_forecast_result['number_of_forecast_month'] = current_month 
    daily_forecast_result['Date'] = decision_month 
    Final_forecast_results = Final_forecast_results.append(daily_forecast_result)
    training_window += 1
Final_forecast_results = Final_forecast_results.set_index(['Date'])

### Feature importances
Plot key features across all back-tested model runs (grouped by mean Gini-importance - consider removing features that showed less than x times across all runs)

In [ ]:
# Obtain feature importance value counts in order to take only those features which have shown more than x (currently 10) times
complete_feature_importances['value_count'] = complete_feature_importances.index.value_counts()
complete_feature_importances = complete_feature_importances[complete_feature_importances.value_count >= 10]
complete_feature_importances = complete_feature_importances.sort_values(by='Gini-importance', ascending=False)

In [ ]:
#Subset the features grouped by the mean Gini-importance score of their ocurrence across backtesting model runs
key_drivers = complete_feature_importances.reset_index()
key_drivers = key_drivers.drop(columns=['value_count'])
key_drivers = key_drivers.groupby('index').mean()
key_drivers = key_drivers.sort_values(by='Gini-importance', ascending=False)

# Plot features ranksed by Gini-importance for backtested model
key_drivers = key_drivers.sort_values(by='Gini-importance', ascending=False)
key_drivers.plot(kind='bar')
plt.title("Ranking of Gini-importance for selected features")

In [ ]:
# Write to excel
key_drivers.to_excel(r'Key_Drivers.xlsx', index = True)
key_drivers.to_csv(r'Key_Drivers.csv', index = True)

## Final results:

#### Obtain actuals and predictions columns from the desired forecast

In [ ]:
Final_forecast_results = Final_forecast_results[~Final_forecast_results.index.duplicated(keep='first')]

In [ ]:
#SET COLUMNS TO RUN FINAL PLOTS AND EVALUATION
y_test = Final_forecast_results['actual_2']
y_pred = Final_forecast_results['forecast_result_2']
upper = Final_forecast_results['forecast_upper_2']
lower = Final_forecast_results['forecast_lower_2']

#### Plot results vs. predicted

In [ ]:
# Plot results vs. predicted for each forecast
plt.scatter(y_test, y_pred)
plt.xlabel('Reported')
plt.ylabel('Predicted')
plt.show()

#### MAE on final forecast results, 

In [ ]:
#plot each of the forecast vs actual on line chart and then run the MAE for each of the forecasts 
x = y_test.index
plt.plot(x, upper, color = 'yellow')
plt.plot(x, lower, color = 'yellow')
plt.fill_between(x,lower,upper,interpolate=True,color='yellow')
plt.plot(x, y_pred, color = 'purple')
plt.plot(x, y_test, color = 'green')
plt.show()

regression_results(y_test, y_pred)

#### Binary and Multi-class confusion matrices for directional accuracy, 

In [ ]:
# Build confusion matrix (binary)
cutoff = "---INSERT CUTOFF LIMIT HERE---"                            

###Example:
# cutoff = 0

y_pred_classes = np.zeros_like(y_pred)    # initialise a matrix full with zeros
y_pred_classes[y_pred > cutoff] = 1       # add a 1 if the cutoff was breached

# You have to do the same for the actual values too:
y_test_classes = np.zeros_like(y_pred)
y_test_classes[y_test > cutoff] = 1

# Now run the confusion matrix as before:
confusion = confusion_matrix(y_test_classes, y_pred_classes)
print('Confusion Matrix\n')
print(confusion)

In [ ]:
# # Build confusion matrix (multi-class)
# cutoff_1 = "---INSERT CUTOFF LIMIT HERE---"
# cutoff_2 = "---INSERT CUTOFF LIMIT HERE---"

####Example:
# cutoff_1 = -0.05                             # decide on a cutoff limit
# cutoff_2 = 0.05

# y_pred_classes = np.zeros_like(y_pred)    # initialise a matrix full with zeros
# y_pred_classes[y_pred < cutoff_1] = 1       # add a 1 if the cutoff was breached
# y_pred_classes[(y_pred >= cutoff_1) & (y_pred < cutoff_2)] = 2       # add a 2 if the cutoff was breached
# y_pred_classes[y_pred >= cutoff_2] = 3       # add a 3 if the cutoff was breached


# # you have to do the same for the actual values too:
# y_test_classes = np.zeros_like(y_test)    # initialise a matrix full with zeros
# y_test_classes[y_test < cutoff_1] = 1       # add a 1 if the cutoff was breached
# y_test_classes[(y_test >= cutoff_1) & (y_test < cutoff_2)] = 2       # add a 2 if the cutoff was breached
# y_test_classes[y_test >= cutoff_2] = 3       # add a 3 if the cutoff was breached


# # Now run the confusion matrix as before:
# # confusion = confusion_matrix(y_test_classes, y_pred_classes)
# # print('Confusion Matrix\n')
# # print(confusion)

#### Accuracy, precision, recall, F1 using classes built for confusion matrix

In [ ]:
#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test_classes, y_pred_classes)))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test_classes, y_pred_classes, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test_classes, y_pred_classes, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test_classes, y_pred_classes, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test_classes, y_pred_classes, target_names=['Class 1', 'Class 2'])) #Class 3 if you want to use multi-class confusion matrix

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
fpr, tpr, threshold = metrics.roc_curve(y_test_classes, y_pred_classes)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#### Conversion of target from % target to actual price for MAPE evaluation

In [ ]:
# Pull required columns from final forecast table
price_conversion = Final_forecast_results[["---INSERT FORECAST RESULT, PREDICTED LOWER BOUND, PREDICTED UPPER BOUND, AND ACTUAL FOR EACH TARGET PREDICTED"]]

###Example: (grabs targets for % difference from current price vs. 1-3m and 4-6m median -- in this case the results for 1,2,3 are all the same and likewise for 4,5,6)
# price_conversion = Final_forecast_results[['forecast_result_2', 'forecast_lower_2', 'forecast_upper_2', 'actual_2', 'forecast_result_4', 'forecast_lower_4', 'forecast_upper_4', 'actual_4']]

In [ ]:
# Bring in real the actual current prices to make conversion 
price_conversion.index = pd.DatetimeIndex(price_conversion.index)
price_conversion = pd.merge(price_conversion,target_data_select, how='left', left_index=True, right_index=True)

In [ ]:
# Convert % change targets to prices
price_conversion['predicted_price_1_3'] = price_conversion['settle'] * (1+ price_conversion['forecast_result_2'])
price_conversion['lower_bound_1_3'] = price_conversion['settle'] * (1+ price_conversion['forecast_lower_2'])
price_conversion['upper_bound_1_3'] = price_conversion['settle'] * (1+ price_conversion['forecast_upper_2'])

price_conversion['predicted_price_4_6'] = price_conversion['settle'] * (1+ price_conversion['forecast_result_4'])
price_conversion['lower_bound_4_6'] = price_conversion['settle'] * (1+ price_conversion['forecast_lower_4'])
price_conversion['upper_bound_4_6'] = price_conversion['settle'] * (1+ price_conversion['forecast_upper_4'])

In [ ]:
# Plot actuals and CI for model predicting 1-3
x = price_conversion.index
y_test = price_conversion['Median_1_3']
y_pred = price_conversion['predicted_price_1_3']
upper = price_conversion['upper_bound_1_3']
lower = price_conversion['lower_bound_1_3']

plt.plot(x, upper, color = 'yellow')
plt.plot(x, lower, color = 'yellow')
plt.fill_between(x,lower,upper,interpolate=True,color='yellow')
plt.plot(x, y_pred, color = 'purple')
plt.plot(x, y_test, color = 'green')
plt.show()

In [ ]:
#Plot actuals and CI for model predicting 4-6
x = price_conversion.index
y_test = price_conversion['Median_4_6']
y_pred = price_conversion['predicted_price_4_6']
upper = price_conversion['upper_bound_4_6']
lower = price_conversion['lower_bound_4_6']

plt.plot(x, upper, color = 'yellow')
plt.plot(x, lower, color = 'yellow')
plt.fill_between(x,lower,upper,interpolate=True,color='yellow')
plt.plot(x, y_pred, color = 'purple')
plt.plot(x, y_test, color = 'green')
plt.show()

In [ ]:
# Safeguard: Implement forward/back-fill mean imputation for missing month-years
price_conversion = pd.concat([price_conversion.ffill(), price_conversion.bfill()]).groupby(level=0).mean()

In [ ]:
# Obtain final regression results
regression_results(price_conversion['---INSERT ACTUAL HERE---'], price_conversion['---INSERT PREDICTED HERE---'])
...
...

###Example:
# regression_results(price_conversion['Median_1_3'], price_conversion['predicted_price_1_3'])
# regression_results(price_conversion['Median_4_6'], price_conversion['predicted_price_4_6'])

### Prepare final model results for visualization

In [ ]:
price_conversion = price_conversion.drop(columns=['month', 'year'])

In [ ]:
# Create new dataframe that renames all columns for % change and actual price to legible names
final_results = price_conversion
final_results.columns = ['predicted_pct_delta_current_vs_median_1_3', 'predicted_lower_pct_delta_current_vs_median_1_3', 'predicted_upper_pct_delta_current_vs_median_1_3', 'actual_pct_delta_current_vs_median_1_3', 'predicted_pct_delta_current_vs_median_4_6', 'predicted_lower_pct_delta_current_vs_median_4_6', 'predicted_upper_pct_delta_current_vs_median_4_6', 'actual_pct_delta_current_vs_median_4_6', 'actual_price', 'actual_median_price_1_3', 'actual_median_price_4_6', 'predicted_median_price_1_3', "predicted_lower_median_price_1_3",'predicted_upper_median_price_1_3', 'predicted_median_price_4_6', "predicted_lower_median_price_4_6",'predicted_upper_median_price_4_6']

# Take the last rows (the very last will be used to give 1-3 and 4-6 month forecast)
final_results_last = final_results.tail(3)

In [ ]:
# Calculate percentage ranges for vizualization
def calc_range_probabilities(actual, forecasted_result):
    diff = np.std(actual - forecasted_result)
    deviation = diff * 1.96
    
    print("Standard Deviation is: ")
    print(deviation)
    
    print("Range probability 1 is: ")
    print(scipy.stats.norm(forecasted_result.tail(1), diff).cdf(-0.05))
    
    print("Range probability 2 is: ")
    print(scipy.stats.norm(forecasted_result.tail(1), diff).cdf(0)-scipy.stats.norm(forecasted_result.tail(1), diff).cdf(-0.05))
    
    print("Range probability 3 is: ")
    print(scipy.stats.norm(forecasted_result.tail(1), diff).cdf(0.05)-scipy.stats.norm(forecasted_result.tail(1), diff).cdf(0))
    
    print("Range probability 4 is: ")
    print(1-scipy.stats.norm(forecasted_result.tail(1), diff).cdf(0.05))
    
    range_df = pd.DataFrame()
    range_df['range_1'] = scipy.stats.norm(forecasted_result, diff).cdf(-0.05)
    range_df['range_2'] = scipy.stats.norm(forecasted_result, diff).cdf(0)-scipy.stats.norm(forecasted_result, diff).cdf(-0.05)
    range_df['range_3'] = scipy.stats.norm(forecasted_result, diff).cdf(0.05)-scipy.stats.norm(forecasted_result, diff).cdf(0)
    range_df['range_4'] = 1-scipy.stats.norm(forecasted_result, diff).cdf(0.05)
    
    range_df = range_df.tail(1)
    
    return range_df

# Calculate range probabilities for predictions
INSERT_VARIABLE_NAME_HERE = calc_range_probabilities(final_results_last['INSERT_VARIABLE_NAME_HERE'], final_results_last['INSERT_VARIABLE_NAME_HERE'])

### Examples:
# range_df_1_3 = calc_range_probabilities(final_results_last['actual_pct_delta_current_vs_median_1_3'], final_results_last['predicted_pct_delta_current_vs_median_1_3'])
# range_df_4_6 = calc_range_probabilities(final_results_last['actual_pct_delta_current_vs_median_4_6'], final_results_last['predicted_pct_delta_current_vs_median_4_6'])

In [ ]:
#Build viz table for 1-3m model
final_results_viz_1_3 = final_results_last[['predicted_median_price_1_3', 'predicted_upper_median_price_1_3', 'predicted_lower_median_price_1_3', 'predicted_pct_delta_current_vs_median_1_3']]
final_results_viz_1_3 = final_results_viz_1_3.tail(1)
final_results_viz_1_3['range_1_prob'] = range_df_1_3['range_1'].values
final_results_viz_1_3['range_2_prob'] = range_df_1_3['range_2'].values
final_results_viz_1_3['range_3_prob'] = range_df_1_3['range_3'].values
final_results_viz_1_3['range_4_prob'] = range_df_1_3['range_4'].values
final_results_viz_1_3 = final_results_viz_1_3.append([final_results_viz_1_3]*2,ignore_index=False)
final_results_viz_1_3.columns = ['price', 'upper', 'lower', 'pct_change', 'range_1_prob', 'range_2_prob', 'range_3_prob', 'range_4_prob']

In [ ]:
#Build viz table for 4-6m model
final_results_viz_4_6 = final_results_last[['predicted_median_price_4_6', 'predicted_upper_median_price_4_6', 'predicted_lower_median_price_4_6', 'predicted_pct_delta_current_vs_median_4_6']]
final_results_viz_4_6 = final_results_viz_4_6.tail(1)
final_results_viz_4_6['range_1_prob'] = range_df_4_6['range_1'].values
final_results_viz_4_6['range_2_prob'] = range_df_4_6['range_2'].values
final_results_viz_4_6['range_3_prob'] = range_df_4_6['range_3'].values
final_results_viz_4_6['range_4_prob'] = range_df_4_6['range_4'].values
final_results_viz_4_6 = final_results_viz_4_6.append([final_results_viz_4_6]*2,ignore_index=False)
final_results_viz_4_6.columns = ['price', 'upper', 'lower', 'pct_change', 'range_1_prob', 'range_2_prob', 'range_3_prob', 'range_4_prob']

In [ ]:
final_results_viz = final_results_viz_1_3.append(final_results_viz_4_6)
final_results_viz['pct_change'] = final_results_viz['pct_change'] * 100

###  Write final model results to CSV and Excel for visualization

In [ ]:
# Write final visualization output in PowerBI to CSV and Excel
final_results.to_excel(r'Forecast_Results_Price.xlsx', index = True)
final_results.to_csv(r'Forecast_Results_Price.csv', index = True)

# Write full predictions + price conversions to CSV and Excel
final_results_viz.to_excel(r'Forecast_Results_PowerBI_Viz.xlsx', index = True)
final_results_viz.to_csv(r'Forecast_Results_PowerBI_Viz.csv', index = True)

In [ ]:
final_results_viz

### Write final model results to CSV and Excel for hedging optimization

In [ ]:
# Write to excel
Final_forecast_results.to_excel(r'Forecast Results.xlsx', index = True)
Final_forecast_results.to_csv(r'Forecast_Results.csv', index = True)

# Show final results table
Final_forecast_results

## Conclusion 

Example: The final model's MAPE on the reduced testing data increases/decreases (going from ...% to ...% MAPE on ...-month forecast and from ...% to ...% MAPE on ...-month forecast). The hit rate (precision) on a binary confusion matrix with ... as cutoff for the ...-month model comes to ...%. If training time was the most important factor one might also consider using a reduced dataset for training, however given that we already optimized for speed it would probably make sense to continue using full data.

## Helper Functions

The following functions are used to pre-process the data being used in the model, run model training and evaluation, identify feature importances, visualize results, and record those results.

In [ ]:
#Pre-process specific commodity dataframe to contain only closest contract to spot price per month
def find_closest_to_spot(data: pd.DataFrame):
    data = data.groupby(['price_date']).first().reset_index(drop=False)[['price_date', 'settle']].set_index(['price_date'])
    data = data.reset_index()
    data.price_date = pd.to_datetime(data.price_date)
    data['price_date'] = data.price_date - pd.offsets.MonthBegin(1)
    data = data.set_index('price_date')
    data = data.groupby(pd.Grouper(freq='M')).mean()
    data['month'] = data.index.month
    data['year'] = data.index.year
    data = data.reset_index()
    
#    # Optional code for only considering contracts with certain time till expiration
#     data = data.sort_values(by=['price_date', 'expiration'], ascending=True)
#     data = data.loc[data.days_to_expiration >= 0]
#     data = data.loc[data.days_to_expiration <= 5]
#     data = data.loc[data.groupby(['month', 'year']).days_to_expiration.idxmin()].reset_index(drop=True)
#     data = data.sort_values(by=['year', 'month'], ascending=True)    
    return data

In [ ]:
def preprocess_commodity_date(data, date):
    data = data.loc[data.price_date >= date] 
    return data

In [ ]:
def crop_barchart_data(preprocessed_data:pd.DataFrame):
    preprocessed_data = preprocessed_data[['price_date', 'settle']]
    preprocessed_data['price_date'] = pd.to_datetime(preprocessed_data['price_date'])
    preprocessed_data['month'] = pd.DatetimeIndex(preprocessed_data['price_date']).month
    preprocessed_data['year'] = pd.DatetimeIndex(preprocessed_data['price_date']).year
    preprocessed_data = preprocessed_data.set_index('price_date')
    return preprocessed_data

In [ ]:
#Basic mean imputation strategy for zeros that are actually missing values
def handle_zero(df: pd.DataFrame):
    for col in df.columns:
        df[col] = df[col].replace(0,df[col].mean())
    return df

In [ ]:
def createList(n):
    lst = []
    for i in range(n+1):
        lst.append(i)
    return(lst)

In [ ]:
def feat_bband(data, x, z):
    """Bollinger band."""
    sma = feat_avg(data, x)
    std = feat_std(data, x)
    bband_upper = sma + std * z
    bband_lower = sma - std * z
    return bband_upper, bband_lower
def feat_rsi(data, x):
    """Relative strength index."""
    rsi = RSI(data, timeperiod=14)
    return rsi
def feat_macd(data, x):
    """Moving average convergence divergence."""
    macd, macdsignal, macdhist = MACD(data, fastperiod=12, slowperiod=26, signalperiod=9)
    return macd
def feat_avg(data, x):
    """Rolling average."""
    return data.rolling(x, min_periods=1).mean()
def feat_med(data, x):
    """Rolling median."""
    return data.rolling(x, min_periods=1).median()
def feat_min(data, x):
    """Rolling min."""
    return data.rolling(x, min_periods=1).min()
def feat_max(data, x):
    """Rolling max."""
    return data.rolling(x, min_periods=1).max()
def feat_std(data, x):
    """Rolling standard deviation."""
    return data.rolling(x, min_periods=1).std()
def feat_lag(data, x):
    """Lag terms of current value (x days before)."""
    return data.shift(x)
def feat_trend(data, x):
    """Trend defined as return of x days over mean of the same period."""
    return (data - data.shift(x)) / feat_avg(data, x)
def feat_first_deriv_avg(data, x):
    """Rolling avg of 1st derivative."""
    return data.diff().rolling(x, min_periods=1).mean()
def feat_second_deriv_avg(data, x):
    """Rolling avg of 2nd derivative."""
    return data.diff().diff().rolling(x, min_periods=1).mean()
def feat_first_deriv_max(data, x):
    """Rolling max of fst derivative."""
    return data.diff().rolling(x, min_periods=1).max()
def feat_second_deriv_max(data, x):
    """Rolling max of 2nd derivative."""
    return data.diff().diff().rolling(x, min_periods=1).max()
def calc_slope(x):
    """Linear slope helper function."""
    slope = np.polyfit(np.arange(len(x), dtype=float), x, 1)[0]
    return slope
def feat_slope(data, x):
    """Slope of past x days."""
    return data.rolling(x).apply(calc_slope)

def generate_forward_curve(data: pd.DataFrame, fwd_curve_params: Dict):
    """Generate forward curve with given future contracts.
    Args:
        data: master dataframe with forward curve in it
        fwd_curve_params: Dictionary contain underlying contract of future contracts
    Returns:
        data: Dataframe with forwrd slope of underlying commodity
    """
    for product in fwd_curve_params["product"]:
        fwd_curve_point = fwd_curve_params["forward_curve"]
        fwd_curve_col_name = [product + "_{}".format(p) for p in fwd_curve_point]
        sub_data = data[fwd_curve_col_name].copy()
        # calculate the slope of forward curve using continues forward curve data
        data["{}_fwd_slope".format(product)] = sub_data.apply(calc_slope, axis=1)
        data["{}_fwd_slope".format(product)] = (
            data["{}_fwd_slope".format(product)]
            .replace(0, np.nan, inplace=False)
            .fillna(method="ffill")
        )
    return data

In [ ]:
def calculate_features(
    data: pd.DataFrame, features: list(), x_list: list(), index_col="timestamp"
):
    """Summary.
    Args:
        data pd.DataFrame: data that contain all raw data used to generate features
        features List: name of raw data to generate features
        x_list List: lag terms used to generate features
        index_col (str, optional): Index columnDefaults to "timestamp".
    Returns:
        pd.DataFrame: featured master table
    """
    data_to_gen = data
    data_output = pd.DataFrame(data_to_gen[index_col])
    selected_features = features
    test_horizon = x_list
    for f in selected_features:
        for x in test_horizon:
            # calculate moving average (check)
            data_output["feat__{}__mean_{}_months".format(f, x)] = feat_avg(
                data_to_gen[f], x
            )
            # calculate value over mv avg (check)
            data_output["feat__{}__current_over_mean_{}_months".format(f, x)] = (
                data_to_gen[f] - data_output["feat__{}__mean_{}_months".format(f, x)]
            )
            # calculate std (check)
            data_output["feat__{}__std_{}_months".format(f, x)] = feat_std(
                data_to_gen[f], x
            )
            # calculate median (check)
            data_output["feat__{}__median_{}_months".format(f, x)] = feat_med(
                data_to_gen[f], x
            )
            # calculate min (check)
            data_output["feat__{}__min_{}_months".format(f, x)] = feat_min(
                data_to_gen[f], x
            )
            # calculate max (check)
            data_output["feat__{}__max_{}_months".format(f, x)] = feat_max(
                data_to_gen[f], x
            )
            # calculate lag (check)
            data_output["feat__{}__lag_{}_months".format(f, x)] = feat_lag(
                data_to_gen[f], x
            )
            # calculate delta (check)
            data_output["feat__{}__delta_{}_months".format(f, x)] = (
                data_to_gen[f] - data_output["feat__{}__lag_{}_months".format(f, x)]
            )
            # calculate bband (check)
            bband_upper, bband_lower = feat_bband(data_to_gen[f], x, 1.5)
            data_output["feat__{}__bband_upper_{}_months".format(f, x)] = bband_upper
            data_output["feat__{}__bband_lower_{}_months".format(f, x)] = bband_lower
            # calculate trend (check)
            data_output["feat__{}__trend_{}_months".format(f, x)] = (
                data_output["feat__{}__delta_{}_months".format(f, x)]
                / data_output["feat__{}__mean_{}_months".format(f, x)]
            )
            # calculate RSI (check)
            data_output[
                "feat__{}__RSI_{}_months".format(f, x)
            ] = feat_rsi(data_to_gen[f], x)
            # calculate MACD (check)
            data_output[
                "feat__{}__MACD_{}_months".format(f, x)
            ] = feat_macd(data_to_gen[f], x)
            # calculate 1st deriv (check)
            data_output[
                "feat__{}__avg_first_derivative_{}_months".format(f, x)
            ] = feat_first_deriv_avg(data_to_gen[f], x)
            # calculate 2st deriv (check)
            data_output[
                "feat__{}__avg_second_derivative_{}_months".format(f, x)
            ] = feat_second_deriv_avg(data_to_gen[f], x)
            # calculate 1st deriv (check)
            data_output[
                "feat__{}__max_first_derivative_{}_months".format(f, x)
            ] = feat_first_deriv_max(data_to_gen[f], x)
            # calculate 2st deriv (check)
            data_output[
                "feat__{}__max_second_derivative_{}_months".format(f, x)
            ] = feat_second_deriv_max(data_to_gen[f], x)
            # calculate slope
            data_output["feat__{}__slope_{}_months".format(f, x)] = feat_slope(
                data_to_gen[f], x
            )
            # calculate zscore
            data_output["feat__{}__zscore_{}_months".format(f, x)] = (
                data_to_gen[f] - data_output["feat__{}__mean_{}_months".format(f, x)]
            ) / data_output["feat__{}__std_{}_months".format(f, x)]
            data_output = data_output.copy()
    return data_output.copy()